In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.simplefilter("ignore")
#import tensorflow as tf

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

ss_krr = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
train_features_final=pd.concat([pd.get_dummies(train_features.iloc[:,1:4]),train_features.iloc[:,4:]],axis=1)
#print(train_features_final)
test_features_final=pd.concat([pd.get_dummies(test_features.iloc[:,1:4]),test_features.iloc[:,4:]],axis=1)
#test_features_final

In [ ]:
train_features_final1=(train_features_final-train_features_final.mean())/train_features_final.std()
test_features_final1=(test_features_final-test_features_final.mean())/test_features_final.std()

In [ ]:
'''
ans_df=pd.DataFrame()

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

for i in range(len(train_targets.columns)-1):

    X_train, X_test, y_train, y_test = train_test_split(train_pca,train_targets.iloc[:,i+1], test_size=0.03)


    model = keras.Sequential([
    
        keras.layers.Dense(256, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(128, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(64, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(32, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(16, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(8, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(2)
    ])

    #model.summary()

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=50)

    probability_model = tf.keras.Sequential([model,tf.keras.layers.Softmax()])

    predictions = probability_model.predict(test_pca)
    ans_df=pd.concat([ans_df,pd.DataFrame(predictions[:,1])],axis=1)
'''

In [ ]:
#ans_df=pd.DataFrame()
'''
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

def multiprocessing_func(i):
    
    X_train, X_test, y_train, y_test = train_test_split(train_pca,train_targets.iloc[:,i+1], test_size=0.03)


    model = keras.Sequential([
    
        keras.layers.Dense(256, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(128, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(64, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(32, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(16, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(8, activation='relu',input_shape=[len(X_train.columns)]),
        keras.layers.Dense(2)
    ])

    #model.summary()

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=25)

    probability_model = tf.keras.Sequential([model,tf.keras.layers.Softmax()])

    predictions = probability_model.predict(test_pca)
    return(predictions)
    #ans_df=pd.concat([ans_df,pd.DataFrame(predictions[:,1])],axis=1)
'''

In [ ]:
'''
import multiprocessing
pool=multiprocessing.Pool()
result=pool.map(multiprocessing_func, range(len(train_targets.columns)-1))
pool.close()
pool.join()
'''

In [ ]:
#ans_df

In [ ]:
'''
#Logit
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(class_weight="balanced")
final_ans=pd.DataFrame()
#train_targets.iloc[:,1]
for i in range(len(train_targets.columns)-1):
#    train_targets.iloc[:,i+1]
    lr.fit(train_features_final1, train_targets.iloc[:,i+1])
    y_pred = lr.predict_proba(test_features_final1)
    final_ans=pd.concat([final_ans,pd.DataFrame(y_pred).iloc[:,1]],axis=1)
    #print(i)
    
'''

In [ ]:
#train_pca

In [ ]:
'''
#Logit on PCA
combined_pca=pd.concat([train_features_final1.iloc[:,5:],test_features_final1.iloc[:,5:]],axis=0).reset_index().iloc[:,1:]
#from 23814: test data
#combined_pca
from sklearn.decomposition import PCA
pca_combined = PCA(n_components=400)
principalComponents = pca_combined.fit_transform(combined_pca)
#print(principalComponents.shape)
#pca_combined.explained_variance_ratio_.sum()
principalComponents_train=pd.DataFrame(principalComponents[:23814,:])
principalComponents_test=pd.DataFrame(principalComponents[23814:,:])
#principalComponents_test.columns==principalComponents_train.columns
#principalComponents_test
train_pca=pd.concat([train_features_final1.iloc[:,0:5],principalComponents_train],axis=1)
test_pca=pd.concat([test_features_final1.iloc[:,0:5],principalComponents_test],axis=1)


from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(class_weight="balanced")
final_ans=pd.DataFrame()
for i in range(len(train_targets.columns)-1):
    lr.fit(train_pca, train_targets.iloc[:,i+1])
    y_pred = lr.predict_proba(test_pca)
    final_ans=pd.concat([final_ans,pd.DataFrame(y_pred).iloc[:,1]],axis=1)
    #print(i)
'''

In [ ]:
'''
#Xgboost on PCA
#from sklearn.linear_model import LogisticRegression
combined_pca=pd.concat([train_features_final1.iloc[:,5:],test_features_final1.iloc[:,5:]],axis=0).reset_index().iloc[:,1:]
#from 23814: test data
#combined_pca
import time
import xgboost as xgb
from sklearn.decomposition import PCA
pca_combined = PCA(n_components=400)
principalComponents = pca_combined.fit_transform(combined_pca)
#print(principalComponents.shape)
#pca_combined.explained_variance_ratio_.sum()
principalComponents_train=pd.DataFrame(principalComponents[:23814,:])
principalComponents_test=pd.DataFrame(principalComponents[23814:,:])
#principalComponents_test.columns==principalComponents_train.columns
#principalComponents_test
train_pca=pd.concat([train_features_final1.iloc[:,0:5],principalComponents_train],axis=1)
test_pca=pd.concat([test_features_final1.iloc[:,0:5],principalComponents_test],axis=1)

final_ans=pd.DataFrame()
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.4, learning_rate = 0.075,
                max_depth = 8, alpha = 0.1, n_estimators = 150)
tic=time.time()
for i in range(len(train_targets.columns)-1):
#xg_reg.fit(X_train,y_train)
    xg_reg.fit(train_pca,train_targets.iloc[:,i+1])
    preds = xg_reg.predict_proba(test_pca)
    final_ans=pd.concat([final_ans,pd.DataFrame(preds).iloc[:,1]],axis=1)
    if(i%25==0):
        print ((time.time()-tic)/60) 
'''

In [ ]:
# Xgboost on All data
#from sklearn.linear_model import LogisticRegression
import time
import xgboost as xgb

final_ans=pd.DataFrame()
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.075,
                max_depth = 8, alpha = 0.12, n_estimators = 200)
#tic=time.time()

for i in range(len(train_targets.columns)-1):
    xg_reg.fit(train_features_final1,train_targets.iloc[:,i+1])
    preds = xg_reg.predict_proba(test_features_final1)
    final_ans=pd.concat([final_ans,pd.DataFrame(preds).iloc[:,1]],axis=1)
#    if(i%25==0):
#        print ((time.time()-tic)/60) 


In [ ]:
final_ans 

In [ ]:
#final_ans[final_ans<0.005]=0
#final_ans

In [ ]:
final_ans1=pd.concat([ss_krr.sig_id,final_ans],axis=1)

In [ ]:
final_ans1.columns=ss_krr.columns

In [ ]:
#final_ans1

In [ ]:
final_ans1.to_csv('./submission.csv',index=False)

In [ ]:
df=pd.read_csv("submission.csv")
print(df.shape)
df.head()